# Experiments

#### Same abundance among species

* Objective: community growth
* Objective: community growth + production of metabolite of interest
* Objective: community growth + production of metabolite of interest + anaerobic conditions

#### Use abundances from Amils et al. 2023

* A community model for each depth
* A community model merging the maximum at all depths